In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np
from tqdm import tqdm_notebook as tqdm

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 7, 1, bias=False)
        self.conv2 = nn.Conv2d(32, 128, 5, 1, bias=False)
        self.fc1 = nn.Linear(3*3*128, 10)

    def forward(self, x):
        x = F.leaky_relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.leaky_relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 3*3*128)
        x = self.fc1(x)
        
        return F.log_softmax(x, dim=1)

In [3]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data',
                   train=True,
                   download=True,
                   transform=transforms.ToTensor()
                  ),
    batch_size=128
)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, 
                   transform=transforms.ToTensor()
                  ),
    batch_size=128
)

In [4]:
#torch.backends.cudnn.deterministic = True
#torch.backends.cudnn.benchmark = False

In [5]:
kernels_7x7 = []
kernels_5x5 = []

In [6]:
for n_model in tqdm(range(1,300)):
    
    torch.manual_seed(n_model)
    
    model = Net()
    model.cuda()
    optimizer = optim.Adam(model.parameters(), weight_decay=1e-2)
    
    for epoch in range(1,4):      
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()

        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))

        model.eval()
        test_loss = 0
        correct = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.cuda(), target.cuda()
                output = model(data)
                test_loss += F.nll_loss(output, target, reduction='sum').item()
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)

        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))

    if 100. * correct / len(test_loader.dataset) > 93:
        kernels_7x7.append(list(model.parameters())[0].cpu().data.numpy())
        kernels_5x5.append(list(model.parameters())[1].cpu().data.numpy())

        np.save('./kernels/kernels_7x7.npy', np.vstack(kernels_7x7))

        np.save('./kernels/kernels_5x5.npy', np.vstack(kernels_5x5))

Train Epoch: 1 [44928/60000 (100%)]	Loss: 0.272358

Test set: Average loss: 0.1593, Accuracy: 9503/10000 (95%)

Train Epoch: 2 [44928/60000 (100%)]	Loss: 0.252354

Test set: Average loss: 0.1148, Accuracy: 9667/10000 (97%)

Train Epoch: 3 [44928/60000 (100%)]	Loss: 0.258932

Test set: Average loss: 0.0888, Accuracy: 9757/10000 (98%)

Train Epoch: 1 [44928/60000 (100%)]	Loss: 0.287399

Test set: Average loss: 0.1612, Accuracy: 9493/10000 (95%)

Train Epoch: 2 [44928/60000 (100%)]	Loss: 0.262918

Test set: Average loss: 0.1317, Accuracy: 9594/10000 (96%)

Train Epoch: 3 [44928/60000 (100%)]	Loss: 0.254681

Test set: Average loss: 0.0961, Accuracy: 9725/10000 (97%)

Train Epoch: 1 [44928/60000 (100%)]	Loss: 0.283824

Test set: Average loss: 0.1655, Accuracy: 9463/10000 (95%)

Train Epoch: 2 [44928/60000 (100%)]	Loss: 0.264200

Test set: Average loss: 0.1325, Accuracy: 9598/10000 (96%)

Train Epoch: 3 [44928/60000 (100%)]	Loss: 0.250631

Test set: Average loss: 0.1089, Accuracy: 9681/10000